In [1]:
import numpy as np

# Heplper Func

In [2]:
def softmax(x):
    """
    x -- Shape of (n_classes, batch_size)
    """
        
    x_shifted = x - np.max(x, axis=0, keepdims=True)
    e_x = np.exp(x_shifted)
    probs = e_x / e_x.sum(axis=0, keepdims=True)
    return probs

In [3]:
def sigmoid(x):
    x_clipped = np.clip(x, -500, 500)
    s = 1 / (1 + np.exp(-x_clipped))
    return s

In [4]:
def pad_sequences(sequences, max_len=None, padding_value=0):
    """
    Pads a list of sequences to the same length.

    Arguments:
    sequences -- list of numpy arrays of shape (embedding_size, seq_len_i)
    max_len -- int, maximum length to pad to. If None, use the longest sequence in the batch.
    padding_value -- value to pad with, default 0 (usually for PAD token)

    Returns:
    padded_seqs -- numpy array of shape (embedding_size, batch_size, max_len)
    mask -- numpy array of shape (max_len, batch_size), 1 for valid tokens, 0 for PAD
    """
    
    batch_size = len(sequences)
    embedding_size = sequences[0].shape[0]

    if max_len is None:
        max_len = max(seq.shape[1] for seq in sequences)

    padded_seqs = np.full((embedding_size, batch_size, max_len), padding_value)
    mask = np.zeros((max_len, batch_size))

    for i, seq in enumerate(sequences):
        seq_len = seq.shape[1]
        padded_seqs[:, i, :seq_len] = seq
        mask[:seq_len, i] = 1

    return padded_seqs, mask

# 1. RNN

### 1.1. RNN Forward

You can think of the recurrent neural network as the repeated use of a single cell. First, you'll implement the computations for a single time step. The following figure describes the operations for a single time step of an RNN cell: 

<img src="images/rnnCell_forward.png" style="width:100%;height:auto;">
<caption><center><font color='purple'><b>Figure 2</b>: basic RNN cell. Takes as input $x^{\langle t \rangle}$ (current input) and $a^{\langle t - 1\rangle}$ (previous hidden state containing information from the past), and outputs $a^{\langle t \rangle}$ which is given to the next RNN cell and also used to predict $\hat{y}^{\langle t \rangle}$ 
</center></caption>

In [5]:
def rnnCell_forward(xt, a_prev, parameters, mask_t=None):
    """
    Implements a single forward step of the RNN cell

    Arguments:
    xt -- Input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- Python dictionary containing:
                    Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                    Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                    Ba --  Bias, numpy array of shape (n_a, 1)
                    Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                    By -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    mask -- (optional) Binary mask of shape (1, m), 1 for valid tokens, 0 for PAD
    
    Returns:
    at -- next hidden state at timestep "t", numpy array of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (at, a_prev, xt, parameters)
    """
    
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    Ba = parameters["Ba"]
    By = parameters["By"]
    
    at = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + Ba)  # output_shape(n_a, m)

    if mask_t is not None:
        at = mask_t * at + (1 - mask_t) * a_prev  # keeps the previous hidden state in PAD position
    
    yt_pred = softmax(np.dot(Wya, at) + By)  # output_shape(n_y, m)
    
    # assuming n_y is vocab_size (n_y = n_x = vocab_size) to simplify this homework
    # the output at each timestamp must be in the shape of (vocab_size, batch_size) to predict the next token, you can do another layer of linear projection with W(vocab_size, n_y)
    
    cache = (at, a_prev, xt, parameters, mask_t)
    return at, yt_pred, cache

- A recurrent neural network (RNN) is a repetition of the RNN cell that you've just built. 
    - If your input sequence of data is 10 time steps long, then you will re-use the RNN cell 10 times 
- Each cell takes two inputs at each time step:
    - $a^{\langle t-1 \rangle}$: The hidden state from the previous cell
    - $x^{\langle t \rangle}$: The current time step's input data
- It has two outputs at each time step:
    - A hidden state ($a^{\langle t \rangle}$)
    - A prediction ($y^{\langle t \rangle}$)
- The weights and biases $(W_{aa}, W_{ax}, b_{a}, W_{ay}, b_{y})$ are re-used each time step 
    - They are maintained between calls to `rnnCell_forward` in the 'parameters' dictionary

<img src="images/rnnSeq_forward.png" style="width:100%;height:auto;">
<caption><center><font color='purple'><b>Figure 3</b>: Basic RNN. The input sequence $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle Tx \rangle})$  is carried over $Tx$ time steps. The network outputs $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle Tx \rangle})$. </center></caption>

In [6]:
def rnn_forward(x, a0, parameters, mask=None):
    """
    Implements forward propagation through every timestep for RNNs

    Arguments:
    x -- Input data for every timestep (padded), numpy array of shape (n_x, m, Tx).
    a0 -- Initial hidden state, numpy array of shape (n_a, m)
    parameters -- Python dictionary containing: Waa, Wax, Wya, Ba, By
    mask -- (Optional) Binary mask of shape (Tx, m), 1 for valid tokens, 0 for PAD

    Returns:
    a -- Hidden states for every timestep, numpy array of shape (n_a, m, Tx)
    y_pred -- Predictions for every timestep, numpy array of shape (n_y, m, Tx)
    caches -- a tuple of caches for every timestep needed for the backward pass
    """
    
    n_x, m, Tx = x.shape  # Tx is the length of all inputs in batch
    n_y, n_a = parameters["Wya"].shape

    # Storage
    a = np.zeros((n_a, m, Tx))
    y_pred = np.zeros((n_y, m, Tx))
    caches = []

    # Temp Storage
    a_prev = a0

    # Loop over timestep
    for t in range(Tx):
        # Get mask at timestep t if exists
        mask_t = None
        if mask is not None:
            mask_t = mask[t].reshape(1, -1) # shape(1, m)
        
        # Forward through cell
        at, yt_pred, cache = rnnCell_forward(x[:,:,t], a_prev, parameters, mask_t)
        
        # Store
        a[:,:,t] = at
        y_pred[:,:,t] = yt_pred
        caches.append(cache)
        
        # Reset hidden state
        a_prev = at
    
    return a, y_pred, caches

In [7]:
def compute_loss(y_pred, y_true, mask=None):
    """
    Computes the cross-entropy loss for a sequence of predictions, with optional mask.

    Arguments:
    y_pred -- numpy array of shape (n_y, m, Tx), predicted probabilities (after softmax)
    y_true -- numpy array of shape (n_y, m, Tx), one-hot true labels
    mask -- numpy array of shape (Tx, m), 1 for valid tokens, 0 for PAD

    Returns:
    loss -- scalar, average cross-entropy loss over valid tokens
    """

    n_y, m, Tx = y_pred.shape
    
    epsilon = 1e-8 # avoid log(0)
    y_pred_clipped = np.clip(y_pred, epsilon, 1-epsilon)

    ce_loss = (-y_true) * np.log(y_pred_clipped) # output_shape(n_y, m, Tx)
    ce_loss = np.sum(ce_loss, axis=0) # output_shape(m, Tx) => loss for sample i at timestep t

    if mask is not None:
        ce_loss = ce_loss * mask.T
        total_tokens = np.sum(mask)
    else:
        total_tokens = m * Tx

    # Loss averaged by batch and timestep
    avg_loss = np.sum(ce_loss) / total_tokens

    return avg_loss

### 1.2. RNN Backward

Begin by computing the backward pass for the basic RNN cell. Then, in the following sections, iterate through the cells.

<img src="images/rnnCell_Backward_equations.png" alt="RNN Cell Backward Pass" style="width:100%; height:auto;">
<br>
<caption><center><font color='purple'><b>Figure 6</b>: The RNN cell's backward pass. Just like in a fully-connected neural network, the derivative of the cost function $J$ Backpropagates through the time steps of the RNN By following the chain rule from calculus. Internal to the cell, the chain rule is also used to calculate $(\frac{\partial J}{\partial W_{ax}},\frac{\partial J}{\partial W_{aa}},\frac{\partial J}{\partial b})$ to update the parameters $(W_{ax}, W_{aa}, b_a)$. The operation can utilize the cached results from the forward path. </center></caption>


<img src="images/rnnCell_Backward.png" style="width:800px;height:500px;"> <br>
<caption>
    <center>
        <font color='purple'><b>Figure 7</b>: This implementation of <code>rnnCell_backward</code> doesn't include the output dense layer and softmax.  
    </center>
</caption>

In [8]:
def rnnCell_backward(da_next, dz_t, cache):
    """
    Implements the backward pass for the RNN cell (single time step).

    Arguments:
    da_next -- Gradient of loss with respect to the next hidden state at timestep "t", of shape (n_a, m)
    dz_t -- Gradient of loss with respect to the prediction before softmax at timestep "t", of shape (n_y, m)
    cache -- Tuple of values needed for the backward pass, contains (at, a_prev, xt, parameters, mask_t)

    Returns:
    gradients -- Python dictionary containing:
                    dxt -- Gradients of input data, of shape (n_x, m)
                    da_prev -- Gradients of previous hidden state, of shape (n_a, m)
                    dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                    dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                    dBa -- Gradients of bias vector, of shape (n_a, 1)
                    dWya  -- Gradient w.r.t hidden-to-output weights Wya, shape (n_y, n_a)
                    dBy   -- Gradient w.r.t output bias By, shape (n_y, 1)
    """
    
    (a_next, a_prev, xt, parameters, mask_t) = cache
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    Ba = parameters["Ba"]
    By = parameters["By"]

    # Mask here to avoid updating Wy, By at padding positions 
    if mask_t is not None:
        dz_t = dz_t * mask_t # (n_y, m) @ (1, m)
        
    # Gradients from the output at each timestep
    # You can compute those gradients in "rnn_backward" using vectorization, but I put it here for clarity
    dWya = np.dot(dz_t, a_next.T) # (n_y, m) @ (n_a, m).T
    dBy = np.sum(dz_t, axis=1, keepdims=True) # (n_y, 1)
    da_next_from_z = np.dot(Wya.T, dz_t) # (n_y, n_a).T @ (n_y, m)

    # Total gradient for a_next
    da_total = da_next + da_next_from_z # (n_a, m)
    
    # Gradient through the TANH function that computes a_next
    dtanh = da_total * (1 - a_next**2)  # (n_a, m)

    # Mask here to avoid updating Wa, Ba at padding positions
    if mask_t is not None:
        dtanh = dtanh * mask_t  # (n_a, m) @ (1, m)

    # Gradients for parameters that compute a_next
    dWax = np.dot(dtanh, xt.T)  # (n_a, m) @ (n_x, m).T
    dWaa = np.dot(dtanh, a_prev.T)  # (n_a, m) @ (n_a, m).T
    dBa = np.sum(dtanh, axis=1, keepdims=True)  # (n_a, 1)

    # Gradients for input
    dxt = np.dot(Wax.T, dtanh)  # (n_a, n_x).T  @ (n_a, m)
    da_prev = np.dot(Waa.T, dtanh)  # (n_a, n_a).T @ (n_a, m)

    # Mask here to avoid affecting the previous time-step
    if mask_t is not None:
        da_prev = da_prev * mask_t
        dxt = dxt * mask_t
    
    gradients_t = {
        "dxt": dxt, 
        "da_prev": da_prev, 
        "dWax": dWax, 
        "dWaa": dWaa,
        "dBa": dBa, 
        "dWya": dWya, 
        "dBy": dBy
    }
    return gradients_t

In [9]:
def rnn_backward(y_pred, y_true, caches):
    (a_next, a_prev, xt, parameters, _) = caches[0]
    (n_y, m, Tx) = y_pred.shape
    n_x = xt.shape[0]
    n_a = a_next.shape[0]

    # Storage
    dx = np.zeros((n_x, m, Tx))
    dWax = np.zeros_like(parameters["Wax"])
    dWaa = np.zeros_like(parameters["Waa"])
    dBa = np.zeros_like(parameters["Ba"])
    dWya = np.zeros_like(parameters["Wya"])
    dBy = np.zeros_like(parameters["By"])

    # Temp Storage
    da_t = np.zeros((n_a, m)) # the last timestep da is zero

    dz = y_pred - y_true # (n_y, m, Tx)
    # da_from_output = np.tensordot(Wya.T, dz, axes=([1], [0]))  # (n_y, n_a).T @ (n_y, m, Tx) = (n_a, m, Tx)
    # ignored because it's already handled in each cell, although that way is not optimal because we can utilize vectorization
    
    # Backward
    for t in reversed(range(Tx)):
        cache = caches[t]
        gradients_t = rnnCell_backward(da_t, dz[:,:,t], cache)
        
        # Store
        dx[:,:,t] = gradients_t["dxt"]
        dWax += gradients_t["dWax"]
        dWaa += gradients_t["dWaa"]
        dBa += gradients_t["dBa"]
        dWya += gradients_t["dWya"]
        dBy += gradients_t["dBy"]
        
        # Reset the gradient of the hidden state to the previous one
        da_t = gradients_t["da_prev"]

    da0 = da_t

    # Average gradients over batch size
    # Sometimes people also normalize by Tx to avoid very large gradients for long sequences, but it’s optional.
    dWax /= m
    dWaa /= m
    dBa /= m
    dWya /= m
    dBy /= m
    
    gradients = {
        "dx": dx,
        "da0": da0,
        "dWax": dWax,
        "dWaa": dWaa,
        "dBa": dBa,
        "dWya": dWya,
        "dBy": dBy
    }

    # Storing dx if we have another embedding layer needed to train
    # Storing da0 if we use stateful RNN
    return gradients

# 2. LSTM

### 2.1. LSTM Forward

This following figure shows the operations of an LSTM cell.

<img src="images/lstmCell_forward.png" style="width:500;height:400px;">
<caption><center> <b>Figure 4</b>: LSTM-cell. This tracks and updates a "cell state" or memory variable $c^{\langle t \rangle}$ at every time-step, which can be different from $a^{\langle t \rangle}$. </center></caption>

Similar to the RNN example above, you will start by implementing the LSTM cell for a single time-step. Then you can iteratively call it from inside a for-loop to have it process an input with $T_x$ time-steps. 

### About the gates

#### - Forget gate

For the sake of this illustration, lets assume we are reading words in a piece of text, and want use an LSTM to keep track of grammatical structures, such as whether the subject is singular or plural. If the subject changes from a singular word to a plural word, we need to find a way to get rid of our previously stored memory value of the singular/plural state. In an LSTM, the forget gate lets us do this: 

$$\Gamma_f^{\langle t \rangle} = \sigma(W_f[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_f)\tag{1} $$

Here, $W_f$ are weights that govern the forget gate's behavior. We concatenate $[a^{\langle t-1 \rangle}, x^{\langle t \rangle}]$ and multiply by $W_f$. The equation above results in a vector $\Gamma_f^{\langle t \rangle}$ with values between 0 and 1. This forget gate vector will be multiplied element-wise by the previous cell state $c^{\langle t-1 \rangle}$. So if one of the values of $\Gamma_f^{\langle t \rangle}$ is 0 (or close to 0) then it means that the LSTM should remove that piece of information (e.g. the singular subject) in the corresponding component of $c^{\langle t-1 \rangle}$. If one of the values is 1, then it will keep the information. 

#### - Update gate

Once we forget that the subject being discussed is singular, we need to find a way to update it to reflect that the new subject is now plural. Here is the formulat for the update gate: 

$$\Gamma_u^{\langle t \rangle} = \sigma(W_u[a^{\langle t-1 \rangle}, x^{\{t\}}] + b_u)\tag{2} $$ 

Similar to the forget gate, here $\Gamma_u^{\langle t \rangle}$ is again a vector of values between 0 and 1. This will be multiplied element-wise with $\tilde{c}^{\langle t \rangle}$, in order to compute $c^{\langle t \rangle}$.

#### - Updating the cell 

To update the new subject we need to create a new vector of numbers that we can add to our previous cell state. The equation we use is: 

$$ \tilde{c}^{\langle t \rangle} = \tanh(W_c[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_c)\tag{3} $$

Finally, the new cell state is: 

$$ c^{\langle t \rangle} = \Gamma_f^{\langle t \rangle}* c^{\langle t-1 \rangle} + \Gamma_u^{\langle t \rangle} *\tilde{c}^{\langle t \rangle} \tag{4} $$


#### - Output gate

To decide which outputs we will use, we will use the following two formulas: 

$$ \Gamma_o^{\langle t \rangle}=  \sigma(W_o[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_o)\tag{5}$$ 
$$ a^{\langle t \rangle} = \Gamma_o^{\langle t \rangle}* \tanh(c^{\langle t \rangle})\tag{6} $$

Where in equation 5 you decide what to output using a sigmoid function and in equation 6 you multiply that by the $\tanh$ of the previous state. 

In [10]:
def lstmCell_forward(xt, a_prev, c_prev, parameters, mask_t=None):
    """
    Implement a single forward step of the LSTM cell as described in Figure (4)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    c_prev -- Memory state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- Python dictionary containing:
                    Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                    bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                    Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                    bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                    Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                    bc --  Bias of the first "tanh", numpy array of shape (n_a, 1)
                    Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                    bo --  Bias of the output gate, numpy array of shape (n_a, 1)
                    Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                    by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    c_next -- next memory state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, c_next, a_prev, c_prev, xt, parameters, gates)
    """
    
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wy = parameters["Wy"]
    by = parameters["by"]
    
    input_concated = np.vstack((a_prev, xt))  # output_shape(n_a + n_x, m)

    # Compute new cell state
    ft = sigmoid(np.dot(Wf, input_concated) + bf)  # output_shape(n_a, m)
    it = sigmoid(np.dot(Wi, input_concated) + bi)  # output_shape(n_a, m)
    cct = np.tanh(np.dot(Wc, input_concated) + bc) # output_shape(n_a, m)
    c_next = ft * c_prev + it * cct

    # Compute new hidden state
    ot = sigmoid(np.dot(Wo, input_concated) + bo) # output_shape(n_a, m)
    a_next = ot * np.tanh(c_next)

    # Compute output at timestep "t"
    yt_pred = softmax(np.dot(Wy, a_next) + by) # output_shape(n_y, m)

    # Apply mask
    if mask_t is not None:
        a_next = mask_t * a_next + (1 - mask_t) * a_prev
        c_next = mask_t * c_next + (1 - mask_t) * c_prev

    # Cache
    gates = (ft, it, cct, ot)
    cache = (a_next, c_next, a_prev, c_prev, xt, parameters, gates, mask_t)

    return a_next, c_next, yt_pred, cache

Now that you have implemented one step of an LSTM, you can now iterate this over this using a for-loop to process a sequence of $T_x$ inputs. 

<img src="images/lstmSeq_forward.png" style="width:500;height:300px;">
<caption><center> <b>Figure 3</b>: LSTM over multiple time-steps. </center></caption>

In [11]:
def lstm_forward(x, a0, c0, parameters, mask=None):
    """
    Implement the forward propagation of the recurrent neural network using an LSTM-cell described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, Tx).
    a0 -- Initial hidden state, of shape (n_a, m)
    c0 -- Initial cell state, of shape (n_a, m)
    parameters -- Python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        bc -- Bias of the first "tanh", numpy array of shape (n_a, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        bo -- Bias of the output gate, numpy array of shape (n_a, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, Tx)
    c -- Cell states for every time-step, numpy array of shape (n_a, m, Tx)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, Tx)
    caches -- tuple of caches for every time-step needed for the backward pass
    """
    
    n_x, m, Tx = x.shape
    n_y, n_a = parameters["Wy"].shape
    
    # Storage
    a = np.zeros((n_a, m, Tx))
    c = np.zeros((n_a, m, Tx))
    y_pred = np.zeros((n_y, m, Tx))
    caches = []
    
    # Temp Storage
    a_prev = a0
    c_prev = c0

    # Forward
    for t in range(Tx):
        # Get mask at timestep t if exists
        mask_t = None
        if mask is not None:
            mask_t = mask[t].reshape(1, -1)  # shape (1, m)

        # Forward
        a_next, c_next, yt_pred, cache = lstmCell_forward(x[:,:,t], a_prev, c_prev, parameters, mask_t)
        
        # Store
        a[:,:,t] = a_next
        c[:,:,t]  = c_next
        y_pred[:,:,t] = yt_pred
        caches.append(cache)

        # Reset hidden & cell states
        a_prev = a_next
        c_prev = c_next

    return a, c, y_pred, caches   

### 2.2 LSTM Backward

### Gate derivatives

$$d \Gamma_o^{\langle t \rangle} = da_{next}*\tanh(c_{next}) * \Gamma_o^{\langle t \rangle}*(1-\Gamma_o^{\langle t \rangle})\tag{7}$$

$$d\tilde c^{\langle t \rangle} = dc_{next}*\Gamma_u^{\langle t \rangle}+ \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * i_t * da_{next} * \tilde c^{\langle t \rangle} * (1-\tanh(\tilde c)^2) \tag{8}$$

$$d\Gamma_u^{\langle t \rangle} = dc_{next}*\tilde c^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * \tilde c^{\langle t \rangle} * da_{next}*\Gamma_u^{\langle t \rangle}*(1-\Gamma_u^{\langle t \rangle})\tag{9}$$

$$d\Gamma_f^{\langle t \rangle} = dc_{next}*\tilde c_{prev} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * c_{prev} * da_{next}*\Gamma_f^{\langle t \rangle}*(1-\Gamma_f^{\langle t \rangle})\tag{10}$$

### Parameter derivatives 

$$ dW_f = d\Gamma_f^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{11} $$
$$ dW_u = d\Gamma_u^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{12} $$
$$ dW_c = d\tilde c^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{13} $$
$$ dW_o = d\Gamma_o^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{14}$$

To calculate $db_f, db_u, db_c, db_o$ you just need to sum across the horizontal (axis= 1) axis on $d\Gamma_f^{\langle t \rangle}, d\Gamma_u^{\langle t \rangle}, d\tilde c^{\langle t \rangle}, d\Gamma_o^{\langle t \rangle}$ respectively. Note that you should have the `keep_dims = True` option.

### Derivatives with respect to the previous hidden state, previous memory state, and input.

$$ da_{prev} = W_f^T*d\Gamma_f^{\langle t \rangle} + W_u^T * d\Gamma_u^{\langle t \rangle}+ W_c^T * d\tilde c^{\langle t \rangle} + W_o^T * d\Gamma_o^{\langle t \rangle} \tag{15}$$
Here, the weights for equations 13 are the first n_a, (i.e. $W_f = W_f[:n_a,:]$ etc...)

$$ dc_{prev} = dc_{next}\Gamma_f^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} * (1- \tanh(c_{next})^2)*\Gamma_f^{\langle t \rangle}*da_{next} \tag{16}$$
$$ dx^{\langle t \rangle} = W_f^T*d\Gamma_f^{\langle t \rangle} + W_u^T * d\Gamma_u^{\langle t \rangle}+ W_c^T * d\tilde c_t + W_o^T * d\Gamma_o^{\langle t \rangle}\tag{17} $$
where the weights for equation 15 are from n_a to the end, (i.e. $W_f = W_f[n_a:,:]$ etc...)

In [12]:
def lstmCell_backward(da_next, dc_next, cache):
    """
    Implement the backward pass for the LSTM cell (single time-step).

    Arguments:
    da_next -- Gradients of next hidden state, of shape (n_a, m)
    dc_next -- Gradients of next cell state, of shape (n_a, m)
    cache -- cache storing information from the forward pass

    Returns:
    gradients_t -- Python dictionary containing:
                    dxt -- Gradient of input data at time-step t, of shape (n_x, m)
                    da_prev -- Gradient w.r.t. the previous hidden state, numpy array of shape (n_a, m)
                    dc_prev -- Gradient w.r.t. the previous memory state, of shape (n_a, m, T_x)
                    dWf -- Gradient w.r.t. the weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                    dWi -- Gradient w.r.t. the weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                    dWc -- Gradient w.r.t. the weight matrix of the memory gate, numpy array of shape (n_a, n_a + n_x)
                    dWo -- Gradient w.r.t. the weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                    dbf -- Gradient w.r.t. biases of the forget gate, of shape (n_a, 1)
                    dbi -- Gradient w.r.t. biases of the update gate, of shape (n_a, 1)
                    dbc -- Gradient w.r.t. biases of the memory gate, of shape (n_a, 1)
                    dbo -- Gradient w.r.t. biases of the output gate, of shape (n_a, 1)
    """
    
    # gates = (ft, it, cct, ot)
    # cache = (a_next, c_next, a_prev, c_prev, xt, parameters, gates, mask_t)
    (a_next, c_next, a_prev, c_prev, xt, parameters, gates, mask_t) = cache
    (ft, it, cct, ot) = gates
    
    n_x, m = xt.shape
    n_a, _ = a_next.shape

    # Gradients of Loss to Gates
    dot = da_next * np.tanh(c_next) # (n_a, m)
    dct = dc_next + da_next * ot * (1 - np.tanh(c_next)**2)
    dft = dct * c_prev                     
    dit = dct * cct                        
    dcct = dct * it
    
    dft_raw = dft * ft * (1 - ft) # (n_a, m)
    dit_raw = dit * it * (1 - it)
    dcct_raw = dcct * (1 - cct**2)
    dot_raw = dot * ot * (1 - ot)

    # Apply mask to avoid updating parameters in PAD positions
    if mask_t is not None:
        dft_raw = dft_raw * mask_t
        dit_raw = dit_raw * mask_t
        dcct_raw = dcct_raw * mask_t
        dot_raw = dot_raw * mask_t

    # Gradients of Parameters
    input_concated = np.vstack((a_prev, xt))  # (n_a + n_x, m)
    
    dWf = np.dot(dft_raw, input_concated.T) # (n_a, m) @ (n_ax, m).T = (n_a, n_ax)
    dWi = np.dot(dit_raw, input_concated.T)
    dWc = np.dot(dcct_raw, input_concated.T)
    dWo = np.dot(dot_raw, input_concated.T)

    dbf = np.sum(dft_raw, axis=1, keepdims=True) # (n_a, 1)
    dbi = np.sum(dit_raw, axis=1, keepdims=True)
    dbc = np.sum(dcct_raw, axis=1, keepdims=True)
    dbo = np.sum(dot_raw, axis=1, keepdims=True)

    # Gradients of input, hidden state, cell state
    dz = (np.dot(Wf.T, dft_raw) + np.dot(Wi.T, dit_raw) + np.dot(Wc.T, dcct_raw) + np.dot(Wo.T, dot_raw))  # (n_ax, m)
    da_prev = dz[:n_a, :] # (n_a, m)
    dxt = dz[n_a:, :]     # (n_x, m)
    dc_prev = dct * ft    # (n_a, m)

    # Apply mask
    if mask_t is not None:
        da_prev = da_prev * mask_t
        dc_prev = dc_prev * mask_t
        dxt = dxt * mask_t

    # Returns
    gradients_t = {
        "dxt": dxt,
        "da_prev": da_prev,
        "dc_prev": dc_prev,
        "dWf": dWf,
        "dWi": dWi,
        "dWc": dWc,
        "dWo": dWo,
        "dbf": dbf,
        "dbi": dbi,
        "dbc": dbc,
        "dbo": dbo
    }

    return gradients_t

In [13]:
def lstm_backward(y_pred, y_true, caches):
    """
    Implement the backward pass for the LSTM over all timesteps.

    Arguments:
    y_pred -- Predictions for all timesteps, shape (n_y, m, Tx)
    y_true -- True labels, shape (n_y, m, Tx)
    caches -- List of caches from lstm_forward

    Returns:
    gradients -- Python dictionary containing:
                    dx -- Gradient of inputs, shape (n_x, m, Tx)
                    da0 -- Gradient of initial hidden state, shape (n_a, m)
                    dc0 -- Gradient of initial cell state, shape (n_a, m)
                    dWf, dWi, dWc, dWo -- Gradients of LSTM weights
                    dbf, dbi, dbc, dbo -- Gradients of LSTM biases
                    dWy, dby -- Gradients of output layer
    """
    
    (a_next, c_next, a_prev, c_prev, xt, parameters, _, _) = caches[0]
    n_y, m, Tx = y_pred.shape
    n_x = xt.shape[0]
    n_a = a_next.shape[0]

    # Storage
    dx = np.zeros((n_x, m, Tx))
    dWf = np.zeros_like(parameters["Wf"])
    dWi = np.zeros_like(parameters["Wi"])
    dWc = np.zeros_like(parameters["Wc"])
    dWo = np.zeros_like(parameters["Wo"])
    dbf = np.zeros_like(parameters["bf"])
    dbi = np.zeros_like(parameters["bi"])
    dbc = np.zeros_like(parameters["bc"])
    dbo = np.zeros_like(parameters["bo"])
    dWy = np.zeros_like(parameters["Wy"])
    dby = np.zeros_like(parameters["by"])

    # Temp Storage
    da_t = np.zeros((n_a, m))
    dc_t = np.zeros((n_a, m))

    # Gradient of Loss to output before softmax
    dz = y_pred - y_true  # (n_y, m, Tx)    
    
    for t in reversed(range(Tx)):
        cache = caches[t]
        a_next, c_next, a_prev, c_prev, xt, parameters, _, mask_t = cache
        
        dz_t = dz[:,:,t]  # (n_y, m)
        if mask_t is not None:
            dz_t = dz_t * mask_t

        # Gradient from output
        dWy += np.dot(dz_t, a_next.T)  # (n_y, m) @ (n_a, m).T
        dby += np.sum(dz_t, axis=1, keepdims=True)  # (n_y, 1)
        da_from_z = np.dot(parameters["Wy"].T, dz_t)  # (n_y, n_a).T @ (n_y, m)

        # Total da_t from the next timestep & output
        da_total = da_t + da_from_z

        # Backward LSTM cell
        gradients_t = lstmCell_backward(da_total, dc_t, cache)

        # Store
        dx[:, :, t] = gradients_t["dxt"]
        dWf += gradients_t["dWf"]
        dWi += gradients_t["dWi"]
        dWc += gradients_t["dWc"]
        dWo += gradients_t["dWo"]
        dbf += gradients_t["dbf"]
        dbi += gradients_t["dbi"]
        dbc += gradients_t["dbc"]
        dbo += gradients_t["dbo"]

        # Reset current hidden/cell gradients
        da_t = gradients_t["da_prev"]
        dc_t = gradients_t["dc_prev"]

    da0 = da_t
    dc0 = dc_t

    gradients = {
        "dx": dx,
        "da0": da0,
        "dc0": dc0,
        "dWf": dWf / m,
        "dWi": dWi / m,
        "dWc": dWc / m,
        "dWo": dWo / m,
        "dbf": dbf / m,
        "dbi": dbi / m,
        "dbc": dbc / m,
        "dbo": dbo / m,
        "dWy": dWy / m,
        "dby": dby / m
    }
 
    return gradients